In [1]:
import requests
import json
import time
import math
import re
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

### The Configs for Run:

In [4]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

# <font color='red'>Currency Pair</font>

In [5]:
Load_10K_Records=True

currency_pairs = ['EUR_USD','EUR_GBP','EUR_NZD','EUR_AUD','EUR_CHF','EUR_CAD',
                  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD','AUD_USD',
                  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD',
                  'USD_CAD','USD_CHF','CAD_CHF']

currency_pairs = ["EUR_USD"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [6]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [7]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [8]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [9]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [10]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [11]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [12]:
data.shape

(9658, 12)

In [13]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-10-08,17:00:00,17,281,3,Thursday,2015-10-08T17:00:00.000000000Z,8615,1.12824,1.13274,1.12628,1.12760
1,2015-10-08,21:00:00,21,281,3,Thursday,2015-10-08T21:00:00.000000000Z,1597,1.12774,1.12926,1.12762,1.12826
2,2015-10-09,01:00:00,1,282,4,Friday,2015-10-09T01:00:00.000000000Z,1858,1.12830,1.12868,1.12672,1.12843
3,2015-10-09,05:00:00,5,282,4,Friday,2015-10-09T05:00:00.000000000Z,7619,1.12838,1.13470,1.12721,1.13236
4,2015-10-09,09:00:00,9,282,4,Friday,2015-10-09T09:00:00.000000000Z,8508,1.13234,1.13785,1.13153,1.13692


In [14]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9653,2021-12-24,02:00:00,2,358,4,Friday,2021-12-24T02:00:00.000000000Z,2129,1.13254,1.13350,1.13240,1.13304
9654,2021-12-24,06:00:00,6,358,4,Friday,2021-12-24T06:00:00.000000000Z,1857,1.13301,1.13416,1.13283,1.13406
9655,2021-12-24,10:00:00,10,358,4,Friday,2021-12-24T10:00:00.000000000Z,5332,1.13406,1.13436,1.13134,1.13218
9656,2021-12-24,14:00:00,14,358,4,Friday,2021-12-24T14:00:00.000000000Z,5518,1.13214,1.13284,1.13069,1.13152
9657,2021-12-24,18:00:00,18,358,4,Friday,2021-12-24T18:00:00.000000000Z,3142,1.13147,1.13273,1.13037,1.13134


## Simple Moving Average (SMA)

In [15]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [16]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [17]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [18]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9653,2021-12-24,02:00:00,2,358,4,Friday,2021-12-24T02:00:00.000000000Z,2129,1.13254,1.13350,1.13240,1.13304,1.1324,1.1328,1.1306,0.00064,0.00024,0.00244
9654,2021-12-24,06:00:00,6,358,4,Friday,2021-12-24T06:00:00.000000000Z,1857,1.13301,1.13416,1.13283,1.13406,1.1332,1.1329,1.1309,0.00086,0.00116,0.00316
9655,2021-12-24,10:00:00,10,358,4,Friday,2021-12-24T10:00:00.000000000Z,5332,1.13406,1.13436,1.13134,1.13218,1.1329,1.1328,1.1311,-0.00072,-0.00062,0.00108
9656,2021-12-24,14:00:00,14,358,4,Friday,2021-12-24T14:00:00.000000000Z,5518,1.13214,1.13284,1.13069,1.13152,1.1327,1.1326,1.1312,-0.00118,-0.00108,0.00032
9657,2021-12-24,18:00:00,18,358,4,Friday,2021-12-24T18:00:00.000000000Z,3142,1.13147,1.13273,1.13037,1.13134,1.1324,1.1323,1.1314,-0.00106,-0.00096,-0.00006


## Price Range

In [19]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [20]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [21]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
9634,2021-12-24,02:00:00,2,358,4,Friday,2021-12-24T02:00:00.000000000Z,2129,1.13254,1.13350,...,-0.00050,0.00110,0.00046,-0.00064,1,-0.00050,0.00096,0.00064,-0.00001,0.00093
9635,2021-12-24,06:00:00,6,358,4,Friday,2021-12-24T06:00:00.000000000Z,1857,1.13301,1.13416,...,-0.00105,0.00133,0.00010,-0.00123,1,-0.00105,0.00115,0.00123,-0.00050,-0.00001
9636,2021-12-24,10:00:00,10,358,4,Friday,2021-12-24T10:00:00.000000000Z,5332,1.13406,1.13436,...,0.00188,0.00302,0.00218,-0.00084,0,0.00188,0.00030,0.00084,-0.00105,-0.00050
9637,2021-12-24,14:00:00,14,358,4,Friday,2021-12-24T14:00:00.000000000Z,5518,1.13214,1.13284,...,0.00062,0.00215,0.00132,-0.00083,0,0.00062,0.00070,0.00083,0.00188,-0.00105
9638,2021-12-24,18:00:00,18,358,4,Friday,2021-12-24T18:00:00.000000000Z,3142,1.13147,1.13273,...,0.00013,0.00236,0.00139,-0.00097,0,0.00013,0.00126,0.00097,0.00062,0.00188


In [22]:
data.shape

(9639, 30)

## Relative Strength Index (RSI)

In [23]:
delta = data['Close'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
data['RSI'] = 100 - (100/(1 + rs))

## Calculate Average True Range (ATR)

In [24]:
high_low = data['High'] - data['Low']
high_cp = np.abs(data['High'] - data['Close'].shift())
low_cp = np.abs(data['Low'] - data['Close'].shift())
df = pd.concat([high_low, high_cp, low_cp], axis=1)
true_range = np.max(df, axis=1)
data['ATR_14'] = true_range.rolling(14).mean()

# <font color='red'>CANDLE INDEX NUMBER</font>

In [25]:
candle_no = len(data) - 2
candle_no

9637

# Stop Loss & TakeProfit

In [26]:
ATR = data.iloc[candle_no]['ATR_14']
CLOSED_PRICE = data.iloc[candle_no]['Close']
BUY_SL = (CLOSED_PRICE - ATR).round(5)
SELL_SL = (CLOSED_PRICE + ATR).round(5)
BUY_TP = (CLOSED_PRICE + ATR).round(5)
SELL_TP = (CLOSED_PRICE - ATR).round(5)

In [27]:
print('StopLoss for Sell:', SELL_SL)
print('StopLoss for Buy:', BUY_SL)
print('TakeProfit for Sell:', SELL_TP)
print('TakeProfit for Sell:', BUY_TP)

StopLoss for Sell: 1.13382
StopLoss for Buy: 1.12922
TakeProfit for Sell: 1.12922
TakeProfit for Sell: 1.13382
